# This notebook is responsible for downloading and preparing datasets from various source. 

---> 5 videos from google drive (which will be split into frames)  
---> dataset from roboflow


## Define path of the project and the paths where the train/valid sets will be stored

In [22]:
from pathlib import Path
import os

# Path of the project
PROJECT_PATH = Path(os.getcwd()).resolve().parent

# Make the data dir
DATA_PATH = PROJECT_PATH / 'data'
os.makedirs(DATA_PATH, exist_ok=True)

# Make the train dir
TRAIN_PATH = DATA_PATH / 'train'
os.makedirs(TRAIN_PATH, exist_ok=True)

# Make the valid dir
VALID_PATH = DATA_PATH / 'valid'
os.makedirs(VALID_PATH, exist_ok=True)

print(f'Project path -> {PROJECT_PATH}')
print(f'Data Dir path -> {DATA_PATH}')
print(f'Train Dir path -> {TRAIN_PATH}')
print(f'Valid Dir path -> {VALID_PATH}')

Project path -> C:\Users\miki0\OneDrive\Pulpit\Projects\my_projects\FootballAI
Data Dir path -> C:\Users\miki0\OneDrive\Pulpit\Projects\my_projects\FootballAI\data
Train Dir path -> C:\Users\miki0\OneDrive\Pulpit\Projects\my_projects\FootballAI\data\train
Valid Dir path -> C:\Users\miki0\OneDrive\Pulpit\Projects\my_projects\FootballAI\data\valid


## Downlaod videos from google drive